In [7]:
import pandas as pd
import pymongo
import requests
import json
import time

In [2]:
API_KEY = '42aabe7a7b92bfcf410adf2a04849744'
API_KEY2 = '449abe88aa052cf504a6f762cc2a4e64'
client = pymongo.MongoClient("mongodb://101.132.171.223:27017/")
db = client['shoprec']
col = db['restaurant']

In [3]:
def getCoord(x):
    time.sleep(1)
    addr = x['address']
    r = requests.get('https://restapi.amap.com/v3/geocode/geo?', params={
        'key': API_KEY2,
        'address': addr,
        'city': '武汉'
    })
    r.encoding = 'utf-8'
    content = json.loads(r.text)
    if content['count'] == '0':
        return -1
    lon, lat = content['geocodes'][0]['location'].split(',')
    x['lon'] = lon
    x['lat'] = lat
    return x

In [4]:
res = list(col.find())

In [5]:
data = []
non_data = []

In [ ]:
# data = list(map(updateInfo, res))

for i in range(6204, len(res)):
    if res[i]['address'] == '':
        continue
    _data = getCoord(res[i])
    if _data == -1:
        non_data.append(res[i])
        print(i, 'not found')
    else:
        data.append(_data)
        if i % 50 == 0:
            print(i, '/', len(res))

In [8]:
data = pd.read_excel("./restautant.xlsx")

In [9]:
len(data)

9022